# This Scala Program is used to model the pipe line for the Prediction

### 1.0 Importing Libaries 

In [32]:
import  org.apache.spark._ 
import  spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import scala.util.hashing.MurmurHash3
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator  
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{IndexToString, StringIndexer}
import org.apache.spark.sql.DataFrame
import java.io.{File, PrintWriter}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator,CrossValidatorModel}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  
Logger.getLogger("org").setLevel(Level.ERROR)  


import org.apache.spark._
import spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import scala.util.hashing.MurmurHash3
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache....

## 2.0 Starting spark session

In [2]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@187ff3f4


# 3.0 Importing the Cleaned file - Output file from the Wrangling Program

In [3]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load("weatherAUS-clean-FINAL.csv")

data: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [4]:
data.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

#  4.0 Selecting required fields with right data type

### 4.1 Feature Selection

<b>  Based on  Feature selection done using the correlation matrix (separate Scala program attached ) following filelds were identified to be the most  influencing field for   the rainfall on the following day </b>

In [5]:
val df1 = data.selectExpr("Date","Location","MaxTemp","Humidity3pm", "Pressure3pm","Temp3pm","Temp9am","RainToday_Sc",
                         "cast (Month as String) Month","RainTomorrow_Sc")
                        

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [6]:
df1.count()

res2: Long = 142193


### 4.2 Filtering and forming a particular location data

<b>Australia having very distinct 8 climate zones,no single model will work for all australia  Even within a state  there are multiple climate zones(from alpine to high humid) 
Different models need to be trained for different locations
In this approach,separate models will be created for different locations and saved in the directory for the prediction particular location model will be loaded and prediction will be done
Data used was weatherAUS_YN.csv(before imputing the  null values) as it was  observed  that 
dropping null values are more effective than imputing (In our scenario imputing means blunting the features i.e.  generalising the data)
Wind direction (categorical columns) not tested or used in the modelling as too much of encoding required and crashing the docker
correlation matrix(wrangling file)& trial & Error methods were used to select the features) </b>

### 4.3  Dropping the null values if any before the modeling 

In [7]:
val Loc =df1.na.drop()

Loc: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [8]:
Loc.count()

res3: Long = 130412


<b> We have decided to impute NA with Nearest values ( previous day or next day) or monthly average. Even then for some of the weather stations the data for the whole month is empty . Hence we were not able to impute with suitable values.

Hence our group decided that  we will remove NA rows  as it might impact drastically the model efficiency </b>

## 5.0 Modeling  - Data Preparation

<b>The problem is to predict the rain for tomorrow based on Labeled data set . Hence it is Supervised Learning probelm. Moreover since it is to predict whether to rain or not , it is Binary classification. We have decided to model both logistic regression and Random forrest and adopt a model which is best suited for the dataset </b>

### 5.1 Setting the label column(Rain tomorrow)

In [9]:
val df2 = (Loc.select(Loc("RainTomorrow_Sc").as("label"), 
           $"Location",
           $"Date",
          $"MaxTemp", 
         $"Month",
         $"Raintoday_Sc",
          $"Humidity3pm",
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

df2: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


### 5.2 Indexing the month column(categorical)-To capture the seasonal pattern

In [10]:
// Month  which is a categorical variable need  to be indexed and encoded to be used in logistic regression

In [11]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")


Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_6298c04178fd


### 5.3  Encoding the MonIn( column)

In [12]:
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))


Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_681ca8d3e6bf


### 5.4  Setting the assembler

In [13]:
//setting the assembler with required fields for the regression model

In [14]:
 val assembler=(new VectorAssembler().setInputCols(Array(
                                                        "Raintoday_Sc",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_5447d5a4e4df


### 5.5 Splitting the data into test and train sets

In [15]:
val Array(training,test)=df2.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


In [45]:
// Test is generated again one for use with Logistic regression. second one for Random forrest
val Array(training,test_reg)=df2.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test_reg: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


## 6.0  Logistic regression

In [16]:
//Maximum iteration was set to 20,more iterations giving minor improvements in the accuracy but more runtime and docker failures
// as a trade off maximum iteration was set to 10

### 6.1 Create an instance of the Logistic Regression

In [17]:
val lr=new LogisticRegression().setMaxIter(20)
       

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_f64000ebf7c1


### 6.2  Setting the pipeline

In [18]:
val pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_bc44668ec9bb


### 6.3  Cross validation hyper parameter tuning

In [19]:
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.1, 0.01))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_f64000ebf7c1-regParam: 0.1
}, {
	logreg_f64000ebf7c1-regParam: 0.01
})


In [20]:
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)  

cv: org.apache.spark.ml.tuning.CrossValidator = cv_36db993559d5


In [21]:
val cvModel = cv.fit(training)

2020-06-08 22:24:59,760 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2020-06-08 22:24:59,761 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_36db993559d5


### 6.4 Transforming test data with model

In [22]:
val results =cvModel.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


### 6.5  Model Evaluation

In [23]:
val predictions=results.select($"prediction",$"label").as[(Double,Double)].rdd

predictions: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[654] at rdd at <console>:71


In [24]:
val metrics= new MulticlassMetrics(predictions)

metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@5240c6cd


In [25]:
println(metrics.confusionMatrix)

29343.0  1053.0  
6189.0   2706.0  


(1,1) no  pred no  rain
(2,2) yes pred yes rain
(1,2) no pred yes rain 
(2,1)  yes pred no rain 

In [26]:
metrics.accuracy

res8: Double = 0.8156829808353058


### 6.6 Saving the model

In [27]:
cvModel.write.overwrite().save("/home/model_Sydney")

### 6.7 Saving the pipeline

In [28]:
pipeline.write.overwrite().save("/home/pipeline_Sydney")

### 6.8  Reloading the model

In [33]:
val modelLoaded=CrossValidatorModel.load("/home/model_Sydney")

modelLoaded: org.apache.spark.ml.tuning.CrossValidatorModel = cv_36db993559d5


### 6.9 Using the model to run on Test Prediction using Test date

In [34]:
// loaded model is applied on dataframe to see the prediction vs label

In [35]:
val dfu= modelLoaded.transform(df2)

dfu: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


In [36]:
df2.show() //Trying to check the actual Vs prediction for the sydney for last 10 years

+-----+--------+-------------------+-------+-----+------------+-----------+-----------+-------+-------+
|label|Location|               Date|MaxTemp|Month|Raintoday_Sc|Humidity3pm|Pressure3pm|Temp9am|Temp3pm|
+-----+--------+-------------------+-------+-----+------------+-----------+-----------+-------+-------+
|    1|  Cairns|2009-01-01 00:00:00|   31.4|    1|           1|       70.0|     1005.6|   28.0|   30.1|
|    0|  Cairns|2009-01-02 00:00:00|   31.6|    1|           1|       70.0|     1006.8|   30.1|   30.5|
|    0|  Cairns|2009-01-03 00:00:00|   31.6|    1|           0|       70.0|     1007.0|   30.8|   30.2|
|    0|  Cairns|2009-01-04 00:00:00|   32.7|    1|           0|       76.0|     1006.6|   27.8|   28.5|
|    1|  Cairns|2009-01-05 00:00:00|   31.6|    1|           0|       87.0|     1005.7|   27.8|   27.0|
|    0|  Cairns|2009-01-06 00:00:00|   32.4|    1|           1|       63.0|     1004.4|   28.1|   31.1|
|    0|  Cairns|2009-01-07 00:00:00|   32.4|    1|           0| 

In [37]:
val testev=dfu.select("Location","Date","RawPrediction","probability","label","prediction").toDF()

testev: org.apache.spark.sql.DataFrame = [Location: string, Date: timestamp ... 4 more fields]


In [38]:
testev.show()

+--------+-------------------+--------------------+--------------------+-----+----------+
|Location|               Date|       RawPrediction|         probability|label|prediction|
+--------+-------------------+--------------------+--------------------+-----+----------+
|  Cairns|2009-01-01 00:00:00|[0.16930336203980...|[0.54222502845036...|    1|       0.0|
|  Cairns|2009-01-02 00:00:00|[0.18251885673507...|[0.54550346235646...|    0|       0.0|
|  Cairns|2009-01-03 00:00:00|[1.48166810914462...|[0.81482440653540...|    0|       0.0|
|  Cairns|2009-01-04 00:00:00|[1.17925308399507...|[0.76481347970267...|    0|       0.0|
|  Cairns|2009-01-05 00:00:00|[0.63547385908114...|[0.65372960486945...|    1|       0.0|
|  Cairns|2009-01-06 00:00:00|[0.51394163530154...|[0.62573003026511...|    0|       0.0|
|  Cairns|2009-01-07 00:00:00|[1.57360966927745...|[0.82829758646805...|    0|       0.0|
|  Cairns|2009-01-08 00:00:00|[1.60898824555743...|[0.83327087023898...|    0|       0.0|
|  Cairns|

# 7.0 Random Forrest Modeling 

<b>Data set prepared in step 5   will be reused for Random Forrest as well </b>


### 7.1 Create Instance of Random Forrest

In [39]:
val rf = new RandomForestClassifier()
      .setLabelCol("label")
      .setFeaturesCol("features")
      .setNumTrees(10)
      .setMaxBins(41)
rf

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_bcce2132f425
res16: org.apache.spark.ml.classification.RandomForestClassifier = rfc_bcce2132f425


### 7.2 Setting the pipeline

In [40]:
val M_pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,rf))

M_pipeline: org.apache.spark.ml.Pipeline = pipeline_d175986af826


### 7.3 Training the Model

In [41]:
val Trg_model=M_pipeline.fit(training)

Trg_model: org.apache.spark.ml.PipelineModel = pipeline_d175986af826


### 7.4 Save the Training Model

In [42]:
Trg_model.write
  .overwrite()
  .save("spark-RandomForrest-model")

### 7.5 Load the model to try with test data for prediction

In [43]:
val RegModel = PipelineModel.load("spark-RandomForrest-model")

RegModel: org.apache.spark.ml.PipelineModel = pipeline_d175986af826


### 7.6 Run Prediction with the test data 

In [46]:
val results =RegModel.transform(test_reg)

results: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


### 7.7 Evaluate the Results

In [47]:
val evaluator = new MulticlassClassificationEvaluator()
      .setLabelCol("label")
      .setPredictionCol("prediction")
      .setMetricName("accuracy")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_7daedef83524


 <b>Check the Accuracy of the Model</b>

In [48]:
val accuracy = evaluator.evaluate(results)

accuracy: Double = 0.8295029396044896


In [ ]:
// future ideas to expand
// creating models to different climate zones
// trying to predict next couple of days rainfall( e.g. 3 days) with the available data
// Trying deep learning neural networks for better models
// Estimate the amount of rainfall